In [1]:
import os, collections, pprint
import boxsdk

# find them in box -> dev console -> <your app> -> configuration
oauth = boxsdk.OAuth2(
  client_id='', 
  client_secret='', 
  access_token='' # developer token (valid for 60 mins)
)

client = boxsdk.Client(oauth)
print('The current user ID is {0}'.format(client.user().get().id))

The current user ID is 9170430252


In [2]:
global root_folder
root_folder = client.folder(folder_id='0')

In [27]:
def match_by_name(folder, name): 
    files = folder.get()['item_collection']['entries']
    for f in files: 
        if f['name'] == name: 
            return f
    return None


def get_item(locator, kind): 
    '''locator: id or path to find the item
    kind: file_id, dir_id, path
    '''
    if kind == 'file_id': 
        try: 
            return client.file(file_id=locator).get()
        except: 
            raise ValueError('Invalid file id')
    elif kind == 'dir_id': 
        try: 
            return client.folder(folder_id=locator).get()
        except: 
            raise ValueError('Invalid folder id')
    elif kind == 'path': 
        locator = os.path.normpath(locator)
        tokens = locator.split(os.sep)
        if len(tokens) == 0: 
            raise ValueError('Invalid path')
        item = root_folder
        for name in tokens: 
            item = match_by_name(item, name)
            if item is None: 
                raise ValueError('Nonexist item: {}'.format(name))
        return item.get()
    else: 
        raise ValueError('Item id or path is required')


def list_folder_remote(folder, recurse=-1): 
    '''folder: a folder (boxsdk.object.folder.Folder) to inspect its content
    recurse: -1 nonrecursive, 0 recursive
    '''
    # TODO: 
    if not isinstance(folder, boxsdk.object.folder.Folder): 
        raise ValueError('Invalid input')
    if recurse == -1: 
        # non-recursive case
        pprint.pprint(folder.get()['item_collection']['entries'])
    else: 
        items = folder.get()['item_collection']['entries']
        pprint.PrettyPrinter(indent=recurse).pprint(items)
        subfolders = [item for item in items if isinstance(item, boxsdk.object.folder.Folder)]
        for subfolder in subfolders: 
            list_folder_remote(subfolder, recurse+1)

def change_working_folder(folder): 
    # TODO: change working directory from root_folder to any specified one
    pass

In [17]:
def upload_files(srcs, dst): 
    '''srcs: a list of paths (strings) to local files
    dst: a target folder (boxsdk.object.folder.Folder)
    '''
    if isinstance(srcs, str): 
        srcs = [srcs]
    for src in srcs: 
        print('Uploading {}'.format(src), end='...')
        name = os.path.basename(src)
        remote = match_by_name(dst, name)
        if remote is not None: 
            ans = input('{} already exists in {}. Update? y/[n]'.format(name, dst))
            if ans.lower() != 'y': 
                print('Skip')
                continue
            else: 
                remote.update_contents(src)
        else: 
            dst.upload(src)
        print('Done')


def make_if_nonexist_remote(folder, name): 
    '''folder: an existing remote folder (boxsdk.object.folder.Folder)
    name: the name (string) of subfolder to check
    '''
    subfolder = match_by_name(folder, name)
    if subfolder is None or not isinstance(subfolder, boxsdk.object.folder.Folder): 
        subfolder = folder.create_subfolder(name)
        print('New folder created: {}'.format(subfolder))
    return subfolder


def upload_folder(src, dst): 
    '''src: a path (string) to a local directory
    dst: a target folder (boxsdk.object.folder.Folder)
    '''
    for root, dirs, files in os.walk(src): 
        folder = make_if_nonexist_remote(dst, os.path.basename(root))
        files = [os.path.join(root, f) for f in files]
        upload_files(files, folder)
        for name in dirs: 
            upload_folder(os.path.join(root, name), folder)
        break


def download_files(srcs, dst): 
    '''srcs: a list of remote files (boxsdk.object.file.File)
    dst: a local directory (string)
    '''
    if isinstance(srcs, str): 
        srcs = [srcs]
    for src in srcs: 
        print('Downloading {}'.format(src), end='...')
        name = os.path.join(dst, src['name'])
        if os.path.exists(name): 
            ans = input('{} already exists in {}. Overwrite? y/[n]'.format(src['name'], dst))
            if ans.lower() != 'y': 
                print('Skip')
                continue
        with open(name, 'wb') as f: 
            content = src.content()
            f.write(content)
        print('Done')


def make_if_nonexist_local(folder, name): 
    '''folder: the path (string) to an existing local folder
    name: the name (string) of subfolder to check
    '''
    path = os.path.join(folder, name)
    print(path)
    if not os.path.exists(path): 
        os.mkdir(path)
        print('New directory created: {}'.format(path))
    return path


def download_folder(src, dst): 
    '''src: a remote folder to be downloaded (boxsdk.object.folder.Folder)
    dst: target local directory (string)
    '''
    dst = make_if_nonexist_local(dst, src['name'])
    items = src.get()['item_collection']['entries']
    files = []
    subfolders = []
    for item in items: 
        if isinstance(item, boxsdk.object.folder.Folder): 
            subfolders.append(item)
        else: 
            files.append(item)
    download_files(files, dst)
    for subfolder in subfolders: 
        print(dst)
        print(subfolder['name'])
        download_folder(subfolder, dst)

In [5]:
list_folder_remote(root_folder)  # explore root folder (non-recursive)

[<Box Folder - 102186168705 (Charles_RawEEG)>,
 <Box Folder - 123366382709 (shared_folder)>,
 <Box File - 723850189780 (auth.py)>,
 <Box File - 182923522650 (Sophia.Self.6.1.17.MP4)>,
 <Box File - 182924490478 (SophiaAudio6.1.17.m4a)>]


In [6]:
upload_folder('./example', root_folder) # upload a folder to box

New subfolder created: <Box Folder - 123463282523 (example)>
Uploading ./example/test.txt...Done
New subfolder created: <Box Folder - 123464573505 (ex1)>
New subfolder created: <Box Folder - 123464494528 (ex2)>
Uploading ./example/ex2/Gull_portrait_ca_usa.jpg...Done
New subfolder created: <Box Folder - 123465372991 (ex21)>
New subfolder created: <Box Folder - 123465147988 (ex22)>
Uploading ./example/ex2/ex22/10x-featured-social-media-image-size.png...Done
New subfolder created: <Box Folder - 123464682731 (ex3)>
New subfolder created: <Box Folder - 123464504088 (ex31)>
Uploading ./example/ex3/ex31/visual-reverse-image-search-v2_intro.jpg...Done
Uploading ./example/ex3/ex31/image-human-brain_99433-298.jpg...Done
New subfolder created: <Box Folder - 123465040142 (ex311)>
New subfolder created: <Box Folder - 123464888166 (ex3111)>


In [20]:
example = get_item('123463282523', 'dir_id') # grasp a remote item
# example = get_item('example', 'path') # equivalently
list_folder_remote(example, recurse=0) # explore it (recursively); I need to make things displayed in a nicer way

0
[<Box Folder - 123464573505 (ex1)>,
<Box Folder - 123464494528 (ex2)>,
<Box Folder - 123464682731 (ex3)>,
<Box File - 724269866121 (test.txt)>]
1
[]
1
[<Box Folder - 123465372991 (ex21)>,
 <Box Folder - 123465147988 (ex22)>,
 <Box File - 724269076750 (Gull_portrait_ca_usa.jpg)>]
2
[]
2
[<Box File - 724269623198 (10x-featured-social-media-image-size.png)>]
1
[<Box Folder - 123464504088 (ex31)>]
2
[ <Box Folder - 123465040142 (ex311)>,
  <Box File - 724268746502 (image-human-brain_99433-298.jpg)>,
  <Box File - 724269568615 (visual-reverse-image-search-v2_intro.jpg)>]
3
[<Box Folder - 123464888166 (ex3111)>]
4
[]


In [21]:
upload_files('./new.py', get_item('example/ex1', 'path')) # upload a file

Uploading ./new.py...Done


In [26]:
download_folder(example, '/home/yongyiw/Downloads')

/home/yongyiw/Downloads/example
/home/yongyiw/Downloads/example
ex1
/home/yongyiw/Downloads/example/ex1
/home/yongyiw/Downloads/example
ex2
/home/yongyiw/Downloads/example/ex2
/home/yongyiw/Downloads/example/ex2
ex21
/home/yongyiw/Downloads/example/ex2/ex21
New directory created: /home/yongyiw/Downloads/example/ex2/ex21
/home/yongyiw/Downloads/example/ex2
ex22
/home/yongyiw/Downloads/example/ex2/ex22
New directory created: /home/yongyiw/Downloads/example/ex2/ex22
/home/yongyiw/Downloads/example
ex3
/home/yongyiw/Downloads/example/ex3
New directory created: /home/yongyiw/Downloads/example/ex3
/home/yongyiw/Downloads/example/ex3
ex31
/home/yongyiw/Downloads/example/ex3/ex31
New directory created: /home/yongyiw/Downloads/example/ex3/ex31
/home/yongyiw/Downloads/example/ex3/ex31
ex311
/home/yongyiw/Downloads/example/ex3/ex31/ex311
New directory created: /home/yongyiw/Downloads/example/ex3/ex31/ex311
/home/yongyiw/Downloads/example/ex3/ex31/ex311
ex3111
/home/yongyiw/Downloads/example/ex3/e